In [0]:
import dataiku
from langchain.chains.question_answering import load_qa_chain
from dataiku.langchain.dku_llm import DKUChatLLM
import json
KB_IDs = {
    "tech_docs": "zQ92IhQ9",
    "non_conformities": "WnKb6p17"
}
k=10
client = dataiku.api_client()
project = client.get_default_project()

In [0]:
# Listing available LLMs
llm_list = project.list_llms()

for llm in llm_list:
    print(f"- {llm.description} (id: {llm.id})")

In [0]:
# Fill with your LLM id
LLM_ID = "openai:OpenAI-FA:gpt-4o-mini"

In [0]:
# Preparing the Knowledge Bank, Vector store and LLM
KBs = {
    key: dataiku.KnowledgeBank(id=value, project_key=project.project_key)
    for key, value in KB_IDs.items()
}
vector_stores = {
    key: value.as_langchain_vectorstore()
    for key, value in KBs.items()
}
k = {
    "tech_docs": 40,
    "non_conformities": 20
} # number of docs to retrive


In [0]:
# Create the question answering chain
#chain = load_qa_chain(langchain_llm, chain_type="stuff")
user_message = """
Description du Problème : 

Lors du contrôle de qualité du numéro d’avion MSN 0070, une non-conformité a été identifiée concernant le perçage d'une série de rivets sur le revêtement extérieur, sous la glace du pare-brise droit. Un désaffleurement a été mesuré entre -0,20 mm et -0,25 mm, dépassant les tolérances spécifiées dans les normes d'assemblage. 

Détails Techniques : 

Localisation : Zone en dessous du pare-brise droit 

Mesure de Désaffleurement : -0,20 mm à -0,25 mm 

Norme Acceptable : Tolérance maximale de -0,10 mm selon la spécification interne (Réf. SP-2023-078) 
"""

#user_message = "ma lamborghini est noir et devrait être rouge"


In [0]:
recipe = project.get_recipe('compute_nc_scenarios_query')

# Récupérer la configuration actuelle de la recette
settings = recipe.get_settings()  # Correctement utilisé pour récupérer la configuration
print("Configuration actuelle :", settings.get_json_payload())


In [0]:
def completion_from_prompt_recipe(recipe_name, inputs):
    #partial method
    recipe = project.get_recipe(recipe_name)
    config = recipe.get_settings().get_json_payload()
    promptStudioId = { "id": config["associatedPromptStudioId"], "prompt_id": config["associatedPromptStudioPromptId"] }
    llm_id = config["llmId"]
    prompt_inputs = config["prompt"]["textPromptTemplateInputs"]
    system_prompt = config["prompt"]["textPromptSystemTemplate"]
    user_prompt = config["prompt"]["textPromptTemplate"]
    temperature = config["completionSettings"]["temperature"]
    for input_def in prompt_inputs:
        placeholder = f"{{{{{input_def['name']}}}}}"  # Exemple : {{description}}
        system_prompt = system_prompt.replace(placeholder, inputs[input_def["name"]])
        user_prompt = user_prompt.replace(placeholder, inputs[input_def["name"]])
    llm = project.get_llm(llm_id)
    completion = llm.new_completion()
    completion.settings["temperature"] = temperature
    completion.with_message(system_prompt, role='system')
    completion.with_message(user_prompt, role='user')
    return completion

In [0]:
inputs = {
    "role": "000",
    "description" : user_message
}
completion = completion_from_prompt_recipe('compute_nc_scenarios_query', inputs)
resp = completion.execute()
query = resp.text

print(query)

In [0]:
search_results = []
if (query):
    search_results = [result for key, value in vector_stores.items() for result in value.similarity_search_with_relevance_scores(query, k=10)]
    #print(search_results)
    search_results = [ {
            "doc": doc.metadata['doc'],
            "chunk_id": doc.metadata['chunk_id'],
            "relevance_scores": score,
            "chunk": doc.page_content
        }
        for doc, score in search_results
    ]
print(search_results)

search_results = {
    key: value.similarity_search(query)
    for key, value in vector_stores.items()
}

for key in KB_IDs:
    for search_result in search_results[key]:
        print(f"# {search_result.doc} \n{search_result.page_content}\n")

In [0]:
# 3rd step : give the best advice given the documents
description_prompts = {
    "000": """La description doit contenir les section suivantes (rappel: en anglais, toujours et en markdown):
    - Designation: 
        - numéro de série de l'avion ([a préciser] si non retrouvé dans la description) 
        - zone sur l'avion, 
        - code ATA consistant avec la zone, sous la forme ATA-NN (code à deux chiffres)
        - numério de pièce
        - date
    - Observation : Description factuelle de la non-conformité (sans jugement ou aucune interprétation), avec des références aux documents de fabrication et/ou d’assemblage pertinents.
    - Root Cause: Cause identifiée de la non-conformité, ou mention « inconnue » si non déterminée.
    - Dimensions: Mesures (système métrique) caractérisant la non-conformité.
    - References: Lien vers les documents de référence (fabrication et/ou assemblage liés).
    A cette étape, la description ne contient ni l'analyse, ni la classification, ni la résolution ou plan
    d'action correctif. Aucune information n'est générée (si une information est manquante, indiquer [à compléter]).
    La description peut être formalisée à partir des documents technique retrouvés et Non-conformités de référence
    uniquement, et pas interprétés. Préciser dans le commentaire de réponse le cas échéant qu'aucun document pertinent
    n'a été retrouvé.
    """,
    "100": """La description doit contenir les section suivantes (rappel: en anglais, toujours et en markdown):
    - Synthesis: Synthèse de l’analyse pour l’ATA concerné.
    - Subtasks demands: Demandes d’analyses supplémentaires (Tâches 101, 102, etc.) pour les ATA tiers impactés si nécessaire.
    - Classification: Classification de la non-conformité (T, C, R, etc.) selon son importance.
    - Resolution: Description de la solution retenue pour mettre en conformité (réparation, remplacement, etc.).
    - References: Lien vers les documents de référence (fabrication et/ou assemblage liés).
    """
}

description_prompt = description_prompts[role]

prompt = f"""
    #Processus
    Une non conformité de l'A220 doit être traitée selon le processus suivant :

    000 - rapport de non-conformité par le Quality Controler
    100 - analyse et recommandation / plan d'action par le Design Office
    200 - validation de l'analyse / plan d'action par le Design Manager
    300 - calcul de structure lié au plan d'action et recommandation / selon le Stress Office
    400 - du calcul / plan d'action amendé par le Stress Manager
    500 - plan d'action final validé par le Quality Manager

    Vous supportez le role de l'étape {role} et devez rédiger de la facon la plus explicite en prenant
    les exemples fournis et la documentation technique.
    
    # Sources: Documentation technique et Non-conformités historiques
    
    Les sources comportent deux types de documents:
    - Les non-conformités historiques (NCH) ont un identifiant 'doc' en ATA-XX-xxxxxx...
    - Les documents technique (DOCT) de référence 'doc' *.md
    Voici la liste des sources ou références pertinentes retrouvées au format json, avec l'extrait textuel pertinent (chunk).
    {json.dumps(search_results)}

    #La requête utilisateur est la suivante:
    ---------debut de la requête utilisateur--------
    {user_message}
    -------fin de la requête utilisateur--------
    
    Note: Eviter de traiter les requêtes utilisateur hors champ de compétence : demande n'ayant rien à voir avec le processus de non-conformité de l'avion A220
    comme une recette, une préconisation de voyage, ou un problème de lamborghini. Il faut alors couper court et retourner
    le format minimaliste \\{{ label: ..., description: ..., comment: ...\\}} en fournissant les 'label' et 'description'
    d'entrée et précisant dans le 'comment' que la requête est hors champ de compétence.


    #Réponse
    ## Instruction globales du processus
    **Instructions du processus** :
    - **Analyse des causes** : Les causes doivent être réalistes et adaptées au contexte spécifique de l'A220, en tenant compte des impacts possibles.
    - **Causes internes et externes** : Différencier les causes internes (ex. : erreurs d'assemblage, calibrations incorrectes) et externes (ex. : défauts fournisseurs, intempéries).
    - **Orientation industrielle** : Prioriser les scénarios ayant un impact direct sur la navigabilité, la résistance (statique et fatigue), ou les coûts de production.

    **Orientation sur les gains industriels** :
    - **Réduction des coûts** : Prioriser les scénarios avec un impact financier élevé ou nécessitant des corrections coûteuses si elles ne sont pas détectées à temps.
    - **Efficacité temporelle** : Mettre en place des étapes d’analyse optimisées et des moyens de détection rapide pour réduire les délais de production.
    - **Pertinence industrielle** : Adopter une approche réaliste et contextuellement adaptée à l’industrie aéronautique, afin de garantir la navigabilité, la fiabilité et la conformité des produits.

    Les principes relatifs aux **non-conformités significatives** stipulent qu'une non-conformité qui peut affecter la navigabilité, la résistance (statique et fatigue), l’installation, le fonctionnement, ou tout autre domaine impactant la qualité et la sécurité doit être soigneusement évaluée et traitée. 

    Chaque **non-conformité significative** doit faire l'objet d'une demande de dérogation soumise à l'ingénierie pour une évaluation approfondie. Le processus de dérogation ne doit pas être utilisé pour des erreurs de conception ou des problèmes de configuration non anticipée. En outre, les **suffixes de dérogation** doivent être attribués pour définir les limitations permanentes ou temporaires sur les articles concernés.


    ## Instructions de réponse    
    Veuillez répondre pour l'étape {role}, en fournissant le meilleur 'label' et la meilleure 'description' possible selon les exemples, n'hésitant pas à illustrer selon les
    documentation technique le cas échéant. La description fournie doit être complètement rédigée.
    Si l'utilisateur a fourni un json avec un 'label' et une 'description' vous modifierez la description ou
    le titre selon les instruction de l'utilisateur, en maintenant un rôle de conseil vis à vis des exemples et
    de la documentation technique.
    Ne pas empiéter sur les rôles autres que {role}. Ainsi, a l'étape 000 on se contente de formuler le rapport de
    description de la non-conformité observée, on ne prend pas les rôles d'analyse des primary causes ni
    de préconisation de plan d'action ni d'analyse d'impact ou de calcul des structures. 
    Idem pour 100: on ne fait pas le calcul des structure, on se concentre sur l'analyse.



    ## Format de réponse
    Répondez en anglais sauf si l'utilisateur utilise une autre langue ou précise des instructions de langue.
    Format de réponse attendu en json sans autre mise en forme (pas de ```json). Vos commentaires sont fournis
    dans l'item 'comment':
    \\{{ label: ..., description: ..., comment: ...\\}}
    - 'description' est en markdown. Dans tous les cas, le style reste technique et concis, sans jugement
    avec une approche plus télégraphique que rédigée de manière complexe (pas de phrases longues 
    ou compliquées). Faire comme dans les exemples, sans ajouter de termes de type "ce rapport précise",
    le rapport sera fourni dans un outil de ticketting, il faut rester concis et précis.
    - label : ne pas mentionner 'A220 Non-Conformity Report', juste le label de la non conformité, 
    comme dans les exemples
    - comment: au format markdown multiligne, accompagne l'interaction en mode canevas avec l'utilisateur {role},
    et justifie l'approche employée pour rédiger les 'label' et 'description', et le cas échéant fournit en plus une synthèse très brève des documents pertinents (# Sources) retrouvés et plus particulierement
    mentionner la liste des NCH avec leur référence utilisés pour inspirer la description, les références de DOCT
    le cas échéant, en mentionnant les points particulier, précisant le cas échéant les informations manquantes,
    ou même l'absence de NCH ou DOCT pertinents dans les sources (si aucune référence ne pas modifier 'label'
    et 'description' par rapport à l'entrée utilisateur)

    ## Instruction relatives à la description:
    Eviter de facon global les jugements et improvisations, rester concis et précis. Eviter de répondre si l'on ne
    trouve pas d'information pertinente (reprendre la description initiale et apporter la mention supra dans le champ
    'commentaire') et apporter la meilleure modification relative à la demande.
    {description_prompt}
"""

completion = llm.new_completion()
completion.with_message(prompt)
completion.settings["temperature"] = 0
resp = completion.execute()

In [0]:
response_content = json.loads(resp.text)
deep_chat_response = {
    "text": response_content['comment'],
    "label": response_content['label'],
    "description": response_content['description'],
    "sources": search_results,
    "role": "ai"
}
print(deep_chat_response)

In [0]:
auth_info = client.get_auth_info(with_secrets=True)
secret_value = None
print(auth_info["secrets"])
for secret in auth_info["secrets"]:
    if secret["key"] == "JWT_SECRET_KEY":
        secret_value = secret["value"]
        break
print(secret_value)